# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Построить модель с максимально большим значением *accuracy*.

**Цель исследования:**
 * Построить модель для задачи классификации, которая выберет подходящий тариф - Смарт или Ультра, чтобы доля правильных ответов на этой модели была на менее 0.75

**Ход исследования:**

 * Изучение файла с данными
 * Деление исходных данных на обучающую, валидационную и тестовую выборки.
 * Исследование качество разных моделей
 * Проверка качества модели на тестовой выборке.
 * Проверьте модели на вменяемость. 

## Откройте и изучите файл

Для начала импортируем модули, которые нам понадобятся.

In [24]:
# импорт нужных библиотек и модулей
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

Выгрузим данные из csv файла

In [25]:
df = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на первые 10 строк таблицы

In [26]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
 * сalls — количество звонков,
 * minutes — суммарная длительность звонков в минутах,
 * messages — количество sms-сообщений,
 * mb_used — израсходованный интернет-трафик в Мб,
 * is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Продолжим изучать данные. Теперь посмотрим общую информацию о таблице.

In [27]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Получим больше информации при помощи метода describe(), посмотрим на разброс значений по столбцам

In [28]:
# получение разброса значений методом describe()
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<div style="border:solid blue 2px; padding: 20px">

**Выводы**
    
В целом, данные выглядят правдоподобно. Пропусков и аномальных значений нет. 
Имеет смысл только изменить тип данных в столбцах calls и messages на int и в столбце is_ultra на bool

### Предобработка данных

Приведём столбцы к нужным типам данных

In [29]:
# приведение данных в столбце calls к типу int
df['calls'] = df['calls'].astype(int)

In [30]:
# приведение данных в столбце calls к типу int
df['messages'] = df['messages'].astype(int)

In [31]:
# приведение данных в столбце is_ultra к типу bool
df['is_ultra'] = df['is_ultra'].astype(bool)

Проверим, что типы данных изменились

In [32]:
# получение общей информации о данных в таблице users
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   bool   
dtypes: bool(1), float64(2), int64(2)
memory usage: 103.7 KB


<div style="border:solid blue 2px; padding: 20px">

**Выводы**
    
    Данные предобработаны и можно переходить к делению на выборки

## Разбиение данных на выборки

Разделим наши данные на обучающую, валидационную и тестовую выборки. Возьмём их соотношение 3:1:1 как наиболее часто используемый вариант. 
* Получается, что мы разбиваем датасет на 5 частей. Тренировочная выборка - 3/5 датасета, т.е. 60 %
* Тестовая и валидационная получаются по 20%

In [33]:
df_train, df_other = train_test_split(df, test_size=0.4, random_state=42)

Оставшиеся данные разобьём попалам между тестовой и валидационной выборкой

In [34]:
df_test, df_valid = train_test_split(df_other, test_size=0.5, random_state=42)

Посмотрим на размеры получившихся выборок

In [35]:
print("Размер обучающей выборки:", df_train.shape)
print("Размер тестовой выборки:", df_test.shape)
print("Размер валидационной выборки:", df_valid.shape)

Размер обучающей выборки: (1928, 5)
Размер тестовой выборки: (643, 5)
Размер валидационной выборки: (643, 5)


Размеры соотвествуют процентам.

<div style="border:solid blue 2px; padding: 20px">

**Выводы**

    
Итак, наши данные теперь разделены на 3 датасета:
 * 60 % данных приходится на df_train - обучающую выборку
 * 20 % данных приходится на df_test - тестовую выборку
 * 20 % данных приходится на df_valid - валидационную выборку

## Исследуем модели

Теперь создадим отдельные переменные для признаков и целевого признака. Целевой признак у нас is_ultra 

In [36]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

Проверим три алгоритма:
 * Дерево решений
 * Случайный лес
 * Логистическая регрессия

### Дерево решений

Посмотрим на качество модели в зависимости от глубины дерева. Переберём значения от 1 до 15. Обучим модель для каждого из этих значений. Найдём предсказания на валидационной выборке. И посмотрим качество на валидационной выборке для каждого из значений.

In [37]:
best_tree_model = None # переменная для наилучшей модели
best_tree_result = 0 # accuracy наилучшей модели
# цикл, перебирающий значения глубины дерева от 1 до 15
for depth in range(1, 15):
    model_tree = DecisionTreeClassifier(random_state=42, max_depth=depth) 
    model_tree.fit(features_train, target_train) # обучение модели на тренировочной выборке
    predictions_train = model_tree.predict(features_train)
    predictions_valid = model_tree.predict(features_valid) 
    print("depth =", depth)
    print("Точность на обучающей выборке:", accuracy_score(target_train, predictions_train))
    result_tree = accuracy_score(target_valid, predictions_valid)
    print("Точность на валидационной выборке:", result_tree)
    # условие проверки лучшего результата
    if result_tree > best_tree_result:
        best_tree_model = model_tree 
        best_tree_result = result_tree 
print()
print("Accuracy наилучшей модели на валидационной выборке:", best_tree_result)

depth = 1
Точность на обучающей выборке: 0.7479253112033195
Точность на валидационной выборке: 0.7433903576982893
depth = 2
Точность на обучающей выборке: 0.7816390041493776
Точность на валидационной выборке: 0.7916018662519441
depth = 3
Точность на обучающей выборке: 0.7971991701244814
Точность на валидационной выборке: 0.8055987558320373
depth = 4
Точность на обучающей выборке: 0.8086099585062241
Точность на валидационной выборке: 0.8118195956454122
depth = 5
Точность на обучающей выборке: 0.8153526970954357
Точность на валидационной выборке: 0.807153965785381
depth = 6
Точность на обучающей выборке: 0.8226141078838174
Точность на валидационной выборке: 0.8009331259720062
depth = 7
Точность на обучающей выборке: 0.8386929460580913
Точность на валидационной выборке: 0.7978227060653188
depth = 8
Точность на обучающей выборке: 0.8521784232365145
Точность на валидационной выборке: 0.7978227060653188
depth = 9
Точность на обучающей выборке: 0.8635892116182573
Точность на валидационной выб

В дереве решений наилучшее качество показало дерево глубины 4. accuracy там 0.812

### Случайный лес


Посмотрим на качество модели в зависимости от количества деревьев в лесу. Переберём значения от 1 до 20. Обучим модель для каждого из этих значений. Найдём предсказания на валидационной выборке. И посмотрим качество на валидационной выборке для каждого из значений.

In [38]:
best_forest_model = None # переменная для наилучшей модели
best_forest_result = 0 # accuracy наилучшей модели
# цикл, перебирающий значения деревьев от 1 до 20
for est in range(1, 21):
    model_forest = RandomForestClassifier(random_state=42, n_estimators=est) 
    model_forest.fit(features_train, target_train) # обучение модели на тренировочной выборке
    predictions_train = model_forest.predict(features_train)
    predictions_valid = model_forest.predict(features_valid) 
    result_forest = accuracy_score(target_valid, predictions_valid) 
    # условие проверки лучшего результата
    if result_forest > best_forest_result:
        best_forest_model = model_forest 
        best_forest_result = result_forest 
    print("est =", est)
    print("Точность на обучающей выборке:", accuracy_score(target_train, predictions_train))
    print("Точность на валидационной выборке:", result_forest)
print()
print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result)

est = 1
Точность на обучающей выборке: 0.8962655601659751
Точность на валидационной выборке: 0.7309486780715396
est = 2
Точность на обучающей выборке: 0.9014522821576764
Точность на валидационной выборке: 0.7791601866251944
est = 3
Точность на обучающей выборке: 0.9429460580912863
Точность на валидационной выборке: 0.7682737169517885
est = 4
Точность на обучающей выборке: 0.9377593360995851
Точность на валидационной выборке: 0.8009331259720062
est = 5
Точность на обучающей выборке: 0.9642116182572614
Точность на валидационной выборке: 0.7838258164852255
est = 6
Точность на обучающей выборке: 0.9600622406639004
Точность на валидационной выборке: 0.7978227060653188
est = 7
Точность на обучающей выборке: 0.9740663900414938
Точность на валидационной выборке: 0.7884914463452566
est = 8
Точность на обучающей выборке: 0.9704356846473029
Точность на валидационной выборке: 0.7978227060653188
est = 9
Точность на обучающей выборке: 0.983402489626556
Точность на валидационной выборке: 0.7978227060

Мы видим наилучшее значение accuracy при 18 деревьях - 0.82

### Логистическая регрессия

Посмотрим на качество модели логистической регрессии

In [39]:
model_log_reg = LogisticRegression(random_state=42)
model_log_reg.fit(features_train, target_train) # обучение модели на тренировочной выборке
predictions_valid = model_log_reg.predict(features_valid)
result_log_reg = accuracy_score(target_valid, predictions_valid)
print("Точность на валидационной выборке:", result_log_reg)

Точность на валидационной выборке: 0.7682737169517885


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Логистическая регрессия показала точность в 0.768

<div style="border:solid blue 2px; padding: 20px">

**Выводы**

 * Все три модели - дерево решений, случайный лес и логистическая регрессия - прошли порог accuracy 0.75
 * Лучшей глубиной дерева оказалась глубина 4, принесшая accuracy  0.812
 * Результат accuracy логистической регрессии 0.768
 * Наилучшую accuracy даёт модель случайного леса с количеством деревьев - 18. Точность там 0.82

## Проверка модели на тестовой выборке

Проверим модель случайного леса, как модель, принёсшую наибольшее значение accuracy

Уберём из тестовой выборки столбец с целевой переменной и отправим его в переменную target_test

In [40]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Для проверки обучим каждую модель на тестовой выборке и посчитаем accuracy, сравнив результаты с target_test

Объединим обучающую и валидационную выборки и обучим модель на них

In [41]:
features_train_valid = df_train.drop(['is_ultra'], axis=1).append([df_valid.drop(['is_ultra'], axis=1)])
target_train_valid = df_train['is_ultra'].append([df_valid['is_ultra']])

### Случайный лес

In [42]:
# на объединенной выборке
model_best_forest = RandomForestClassifier(random_state=666, n_estimators=4)
model_best_forest.fit(features_train_valid, target_train_valid)
model_best_forest.predict(features_test)
model_best_forest.score(features_test, target_test)

0.7822706065318819

In [43]:
# на обучающей выборке
model_best_forest = RandomForestClassifier(random_state=42, n_estimators=18)
model_best_forest.fit(features_train, target_train)
model_best_forest.predict(features_test)
model_best_forest.score(features_test, target_test)

0.7916018662519441

<div style="border:solid blue 2px; padding: 20px">

**Выводы**
    
 * Наилучший результат на тестовой выборке показала модель случайного леса. accuracy 0.79, что выше порогового значения

## (бонус) Проверка модели на адекватность

Проверим на случайном наборе предсказаний. Для этого понадобится импортировать модуль random

In [44]:
import random
random_predictions = pd.Series(target_test.apply(lambda x: round(random.random())), index=target_test.index)

# оценим точность на этих данных
print('Random predictions accuracy = ', accuracy_score(target_test, random_predictions))

Random predictions accuracy =  0.4976671850699845


In [48]:
const_predictions = pd.Series(target_test.apply(lambda x: 0), index=target_test.index)
# оценим точность на этих данных
print('Const predictions accuracy = ', accuracy_score(target_test, const_predictions))

Const predictions accuracy =  0.6936236391912908


А точность нашей модели получилась 0.79. Итак, мы видим, что наша модель точнее, чем случайно предсказанные данные и чем модель, предсказывающая 0

<div style="border:solid blue 2px; padding: 20px">

**Выводы**
    
Итак, в ходе работы над проектом мы:
 * предобработали данные
 * разделили данные на три выборки в пропорции 60, 20 и 20 процентов
 * исследовали следующие модели
   * дерево решений
   * случайный лес
   * линейная регрессия
 * для каждой модели подобрали гиперпараметры с наилучшим значением accuracy
   *  Лучшей глубиной дерева оказалась глубина 4, принесшая accuracy  0.812
   * Результат accuracy логистической регрессии 0.768
   * Наилучшую accuracy даёт модель случайного леса с количеством деревьев - 18. Точность там 0.82
 * Проверили на тестовой выборке и получили точность 0.79
 * Убедились, что наша модель проходит тест на адекватность и её значение accuracy больше, чем у модели, предсказывающей случайно и предсказывающей 0